In [1]:
# pip install cdlib
# pip install leidenalg 

In [ ]:
import pandas as pd
import ast
from ast import literal_eval
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import rcParams
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from collections import Counter
import os.path
from os import path
import json
import sys
import networkx as nx
import urllib
import numpy as nр
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
from community import community_louvain
import pandas as pd
from IPython.display import display, HTML, display_pretty
from IPython.display import clear_output
import os
from cdlib import algorithms
%matplotlib inline

In [ ]:
def read_data(path, sep):
    """
    Read DataFrame from filie and converts some columns
    Args:
      path (str): file path
      sep (str): separator
    Returns:
      DataFrame

    """
    df = pd.read_csv(path, sep=sep)
    df.loc[:, 'AA'] = df['AA'].apply(lambda x: ast.literal_eval(x))
    df.loc[:, 'W'] = df['W'].apply(lambda x: ast.literal_eval(x))
    return df

In [ ]:
def centrality_ind(filenameInput, info, id_col = 'Id'):
    """
    Compute classical centrality indices and merge to info, save in files
    Args:
      filenameInput (str): filename with edgelist
      info (DataFrame): DataFrame with main info
      id_col (str): name of column with ID
    Returns:
      info (DataFrame) with indices
   
    """

    Gdirect = nx.read_weighted_edgelist(filenameInput,delimiter='\t',create_using=nx.DiGraph)

    print("network loaded" )

    print("Pagerank centrality, directed")

    prc = nx.pagerank(Gdirect, alpha=0.9, personalization=None, max_iter=100, tol=1e-04, nstart=None, weight='weight', dangling=None)

    orig_stdout = sys.stdout

    f = open('out_Pagerank.txt', 'w')

    sys.stdout = f

    for v in Gdirect.nodes():

        print(v,'\t',prc[v])

    sys.stdout = orig_stdout

    f.close()

    prc = pd.DataFrame(prc.items())
    prc.rename(columns = {0:id_col, 1:'PageRank'}, inplace = True)
    prc[id_col] = prc[id_col].astype(float)
    info = info.merge(prc, how = 'left', on = id_col)
    info.loc[:, 'PageRank rank'] = info['PageRank'].rank(method = 'dense', ascending = False)

    print("Eigenvector centrality, directed")

    eigc = nx.eigenvector_centrality(Gdirect, max_iter=1000, tol=1e-04, nstart=None, weight='weight')

    orig_stdout = sys.stdout

    f = open('out_Eigen.txt', 'w')

    sys.stdout = f

    for v in Gdirect.nodes():

        print(v,'\t',eigc[v])

    sys.stdout = orig_stdout

    f.close()

    eigc = pd.DataFrame(eigc.items())
    eigc.rename(columns = {0:id_col, 1:'Eigen'}, inplace = True)
    eigc[id_col] = eigc[id_col].astype(float)
    info = info.merge(eigc, how = 'left', on = id_col)
    info.loc[:, 'Eigen rank'] = info['Eigen'].rank(method = 'dense', ascending = False)


    print("Betweenness centrality, directed")

    b=nx.algorithms.centrality.betweenness_centrality(Gdirect)

    orig_stdout = sys.stdout

    f = open('out_Between.txt', 'w')

    sys.stdout = f

    for v in Gdirect.nodes():

        print(v,'\t',b[v])

    sys.stdout = orig_stdout

    f.close()

    b = pd.DataFrame(b.items())
    b.rename(columns = {0:id_col, 1:'Between'}, inplace = True)
    b[id_col] = b[id_col].astype(float)
    info = info.merge(b, how = 'left', on = id_col)
    info.loc[:, 'Between rank'] = info['Between'].rank(method = 'dense', ascending = False)

    print("done ")

    return info

In [ ]:
def period_ind(id_name, net, info, id_col = 'Id'):
   """
    Compute classical centrality indices by years
    Args:
      id_name (DataFrame): DataFrame with mapping IDs and Names
      net (DataFrame): temporal network with id_col1 (Source), id_col2 (Target), weight and Y (year)
      info (DataFrame): DataFrame with main info
      id_col (str): name of column with ID
    Returns:
      pagerank (DataFrame) with PageRank index
      eigen (DataFrame) with Eigenvector index
      between (DataFrame) with Betweenness index
   
    """
    pagerank = id_name.copy()
    eigen = id_name.copy()
    between = id_name.copy()

    for Y in range(2015, 2022, 1):
        net_Y = net[net['Y'] == Y]
        net_Y[[id_col + '1', id_col + '2', 'weight']].to_csv(f'net_{Y}.csv', sep = '\t', index = False, header = False)
        print(Y)
        
        info_Y = id_name.copy()

        filenameInput = f'net_{Y}.csv'

        Gdirect = nx.read_weighted_edgelist(filenameInput,delimiter='\t',create_using=nx.DiGraph)

        print("network loaded" )

        print("Pagerank centrality, directed")

        prc = nx.pagerank(Gdirect, alpha=0.9, personalization=None, max_iter=100, tol=1e-04, nstart=None, weight='weight', dangling=None)

        prc = pd.DataFrame(prc.items())
        prc.rename(columns = {0:id_col, 1:Y}, inplace = True)
        prc[id_col] = prc[id_col].astype(float)
        pagerank = pagerank.merge(prc, how = 'left', on = id_col)

        print("Eigenvector centrality, directed")

        eigc = nx.eigenvector_centrality(Gdirect, max_iter=1000, tol=1e-04, nstart=None, weight='weight')

        eigc = pd.DataFrame(eigc.items())
        eigc.rename(columns = {0:id_col, 1:Y}, inplace = True)
        eigc[id_col] = eigc[id_col].astype(float)
        eigen = eigen.merge(eigc, how = 'left', on = id_col)

        print("Betweenness centrality, directed")

        b=nx.algorithms.centrality.betweenness_centrality(Gdirect)

        b = pd.DataFrame(b.items())
        b.rename(columns = {0:id_col, 1:Y}, inplace = True)
        b[id_col] = b[id_col].astype(float)
        between = between.merge(b, how = 'left', on = id_col)


    pagerank.fillna(0, inplace = True)
    pagerank = pagerank.merge(info[[id_col, 'PageRank', 'PageRank rank']], on = id_col)    

    eigen.fillna(0, inplace = True)
    eigen = eigen.merge(info[[id_col, 'Eigen', 'Eigen rank']], on = id_col)

    between.fillna(0, inplace = True)
    between = between.merge(info[[id_col, 'Between', 'Between rank']], on = id_col)


    print("done ")
        
    return pagerank, eigen, between

In [ ]:
def graph_statistics(net, isolated, directed = False, dynamic = True):
    """
    Compute graph statistics
    Args:
      net (DataFrame): temporal network with id_col1 (Source), id_col2 (Target), weight and Y (year)
      isolated (DataFrame): isolated vertices by years
      directed (bool): is net directed
      dynamic (bool): if True then compute metrics by all years else by first year
    Returns:
      net_metrics (DataFrame)
    """

    net_metrics = pd.DataFrame()
    net_prev_year = None
    prev_nodes = set()
    prev_edges = set()
    all_prev_nodes = set()
    all_prev_edges = set()

    for year in range(2015, 2022):
        print(year)
        
        if not dynamic:
            net_year = net
            isolated_year = isolated
        else:
            net_year = net[net['Y'] == year]
            isolated_year = isolated[isolated['Y'] == year]

        net_year[['AfId1', 'AfId2', 'weight']].to_csv(f'net_{year}.txt', sep = '\t', index = False, header = False)

        if directed:
            G_year = nx.read_weighted_edgelist(f'net_{year}.txt', delimiter='\t', create_using=nx.DiGraph)
        else:
            G_year = nx.read_weighted_edgelist(f'net_{year}.txt', delimiter='\t')
        
        G_year.add_nodes_from(isolated_year['AfId'])
        

        # Connected components and sizes
        if directed:
            components = nx.weakly_connected_components(G_year)
        else:
            components = nx.connected_components(G_year)
        comp_df = pd.DataFrame()
        i = 1
        for comp in components:
            for au in comp:
                comp_df.loc[au, 'component'] = i
            i += 1
                    
        comp_df = comp_df.reset_index().rename(columns = {'index': 'AfId'})
        cnt_comp = comp_df.groupby('component').count().reset_index().rename(columns = {'AfId':'count'})
       
        # Average path
        if directed:
            strong_components = nx.strongly_connected_components(G_year)
        else:
            strong_components = nx.connected_components(G_year)
            
        

        degrees = [val for (node, val) in G_year.degree()]

        net_metrics.loc['Number of vertices', year] = len(set(net_year['AfId1']) | set(net_year['AfId2']) | set(isolated_year['AfId']))
        net_metrics.loc['Number of edges', year] = len(net_year)
        net_metrics.loc['Number of isolated vertices', year] = len(set(isolated_year['AfId']))
        net_metrics.loc['Proportion of isolated vertices', year] = len(set(isolated_year['AfId'])) / len(set(net_year['AfId1']) | set(net_year['AfId2']))
        net_metrics.loc['Number of components', year] = i - 1
        net_metrics.loc['Average degree', year] = np.mean(degrees)
        net_metrics.loc['Minimum degree', year] = np.min(degrees)
        net_metrics.loc['Maximum degree', year] = np.max(degrees)
        net_metrics.loc['Density', year] = nx.density(G_year)
        
        net_metrics.loc['Size of the biggest component', year] = max(cnt_comp['count'])
        net_metrics.loc['Proportion of vertices in the biggest component', year] = max(cnt_comp['count']) / len(set(net_year['AfId1']) | set(net_year['AfId2']) | set(isolated_year['AfId']))
        net_metrics.loc['Size of the 2nd biggest component', year] = max(cnt_comp[cnt_comp['count'] != max(cnt_comp['count'])]['count'])
        net_metrics.loc['Size of the 3rd biggest component', year] = max(cnt_comp[(cnt_comp['count'] != max(cnt_comp['count'])) & (cnt_comp['count'] != net_metrics.loc['Size of the 2nd biggest component', year])]['count'])
        
        nodes = set(net_year['AfId1']) | set(net_year['AfId2']) | set(isolated_year['AfId'])
        edges = set(net_year[['AfId1', 'AfId2']].to_records()[['AfId1', 'AfId2']].tolist())
        
        if net_prev_year is not None:
            net_metrics.loc['New nodes', year] = len(nodes - all_prev_nodes) / len(nodes)
            net_metrics.loc['New edges', year] = len(edges - all_prev_edges) / len(edges)
            net_metrics.loc['Remaining nodes', year] = len(nodes & prev_nodes) / len(nodes)
            net_metrics.loc['Remaining edges', year] = len(edges & prev_edges) / len(edges)
            net_metrics.loc['Reoccurring nodes', year] = len(nodes & (all_prev_nodes - prev_nodes)) / len(nodes)
            net_metrics.loc['Reoccurring edges', year] = len(edges & (all_prev_edges - prev_edges)) / len(edges)
            

        net_prev_year = net_year.copy()
        prev_nodes = nodes.copy()
        prev_edges = edges.copy()
        all_prev_nodes = all_prev_nodes | nodes
        all_prev_edges = all_prev_edges | edges
        
        if not dynamic:
            break
        

    return net_metrics

In [ ]:
def distance(M1, M2):
    """
    Compute distance between matrices
    Args:
      M1 (numpy array): matrix1
      M2 (numpy array): matrix2
    Returns:
      distance (float) between matrices
    """
    return np.abs(np.array(M1) - np.array(M2)).sum()
    
def net_stability(net, ranks, eps):
    """
    Compute network stability
    Args:
      net (DataFrame): temporal network
      ranks (DataFrame): ranks of vertices by years
      eps (float): epsilon
    Returns:
      stability (list): list of stability
      distance_R (list): list of distances between ranks marices
      distance_C (list): list of distances between matrices of influence
      R (list): rank matrices
      C (list): matrices of influence 
    """

    stability = []
    distance_C = []
    distance_R = []
    C = []
    R = []

    for year in tqdm(range(2015, 2022)):
        net_cur = net[net['Y'] == year]

        # pkl_file = open(f'drive/MyDrive/Stability/R_{eps}_{year}.pkl', 'rb')
        # R_t = pickle.load(pkl_file)
        # pkl_file.close()

        R_t = []
        for i in tqdm(range(len(ranks))):
            row_t = []
            for j in range(len(ranks)):
                rt = int(ranks.loc[i, str(year)] - ranks.loc[j, str(year)] > eps)
                row_t.append(rt)

            R_t.append(row_t)

        R.append(R_t)

        output = open(f'drive/MyDrive/Stability/R_{eps}_{year}.pkl', 'wb')
        pickle.dump(R_t, output)
        output.close()

        # pkl_file = open(f'drive/MyDrive/Stability/C_{eps}_{year}.pkl', 'rb')
        # C_t = pickle.load(pkl_file)
        # pkl_file.close()

        C_t = []

        for i in tqdm(range(len(ranks))):
            af_i = ranks.loc[i, 'AfId']
            
            if af_i not in net_cur['AfId1'].values:
                C_t.append([0] * len(ranks))
                continue

            row_t = []
            for j in range(len(ranks)):
                af_j = ranks.loc[j, 'AfId']
                if af_j not in net_cur['AfId2'].values:
                    row_t.append(0)
                else:
                    net_tmp = net_cur[net_cur['AfId2'] == af_j]

                    if len(net_tmp[net_tmp['AfId1'] == af_i]) == 0:
                        row_t.append(0)
                    else:
                        row_t.append(net_tmp[net_tmp['AfId1'] == af_i]['weight'].values[0] / net_tmp['weight'].sum())
                     
            C_t.append(row_t)

        C.append(C_t)

        output = open(f'drive/MyDrive/Stability/C_{eps}_{year}.pkl', 'wb')
        pickle.dump(C_t, output)
        output.close()

      
    for i in tqdm(range(1, len(C))):
        R_t = R[i-1]
        R_t1 = R[i]
        C_t = C[i-1]
        C_t1 = C[i]
        
        gamma = max(np.max(C_t), np.max(C_t1))
        d_C = distance(C_t, C_t1) / (len(ranks)**2 * gamma)
        distance_C.append(d_C)

        d_R = distance(R_t, R_t1) / (len(ranks) * (len(ranks) - 1))
        distance_R.append(d_R)

        res = np.sqrt((d_R**2 + d_C**2)/2)
        stability.append(res)
    
    return stability, distance_R, distance_C, R, C


In [ ]:
# Community detection functions
# Args:
#   G (networkx graph)
# Returns:
#   comunities (list)

# Louvain's community detection method
def detect_communities_louvain(G):
    partition = community_louvain.best_partition(G)
    communities = list()
    for com in set(partition.values()) :
        list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
        communities.append(sorted(list_nodes))
    return sorted(communities)
  
# Girvan Newman's community detection method
def detect_communities_girvan_newman(G):
    communities = community.girvan_newman(G)
    return sorted(sorted(c) for c in communities)

# Fast Greedy comunity detection method
def detect_communities_greedy(G):
    communities = community.greedy_modularity_communities(G)
    return sorted(sorted(c) for c in communities)

# Label, propogation community detection method
def detect_communities_label_propagation(G):
    communities = list()
    for c in community.label_propagation_communities(G):
        communities.append(sorted(c))
    return sorted(communities)

# Leiden community detection method
def detect_communities_leiden(G):
    communities = algorithms.leiden(G)
    communities = json.loads(communities.to_json())['communities']
    return communities

In [ ]:
def show_save_communities(G, communities, pos, name = 'Communities ', id_col = 'Id',
                          folder = 'community', comm_file = 'communities.csv', image_file = 'image.png', text = ''):
    """
    Draw graph with communties and save communities and image
    Args:
      G (networkx graph): graph
      communities (list): list of comminities in graph
      pos (list): positions of vertices
      name (str): title
      id_col (str): Id column
      folder (str): name of folder to save  
      comm_file (str): name of file to save communities 
      image_file (str): name of file to save image
      text (str): text below image
    """
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', '#F0F8FF', '#FAEBD7', '#00FFFF', '#7FFFD4', '#F0FFFF', '#F5F5DC', '#FFE4C4', '#000000', '#FFEBCD', '#0000FF', '#8A2BE2', '#A52A2A', '#DEB887', '#5F9EA0', '#7FFF00', '#D2691E', '#FF7F50', '#6495ED', '#FFF8DC', '#DC143C', '#00FFFF', '#00008B', '#008B8B', '#B8860B', '#A9A9A9', '#006400', '#A9A9A9', '#BDB76B', '#8B008B', '#556B2F', '#FF8C00', '#9932CC', '#8B0000', '#E9967A', '#8FBC8F', '#483D8B', '#2F4F4F', '#2F4F4F', '#00CED1', '#9400D3', '#FF1493', '#00BFFF', '#696969', '#696969', '#1E90FF', '#B22222', '#FFFAF0', '#228B22', '#FF00FF', '#DCDCDC', '#F8F8FF', '#FFD700', '#DAA520', '#808080', '#008000', '#ADFF2F', '#808080', '#F0FFF0', '#FF69B4', '#CD5C5C', '#4B0082', '#FFFFF0', '#F0E68C', '#E6E6FA', '#FFF0F5', '#7CFC00', '#FFFACD', '#ADD8E6', '#F08080', '#E0FFFF', '#FAFAD2', '#D3D3D3', '#90EE90', '#D3D3D3', '#FFB6C1', '#FFA07A', '#20B2AA', '#87CEFA', '#778899', '#778899', '#B0C4DE', '#FFFFE0', '#00FF00', '#32CD32', '#FAF0E6', '#FF00FF', '#800000', '#66CDAA', '#0000CD', '#BA55D3', '#9370DB', '#3CB371', '#7B68EE', '#00FA9A', '#48D1CC', '#C71585', '#191970', '#F5FFFA', '#FFE4E1', '#FFE4B5', '#FFDEAD', '#000080', '#FDF5E6', '#808000', '#6B8E23', '#FFA500', '#FF4500', '#DA70D6', '#EEE8AA', '#98FB98', '#AFEEEE', '#DB7093', '#FFEFD5', '#FFDAB9', '#CD853F', '#FFC0CB', '#DDA0DD', '#B0E0E6', '#800080', '#663399', '#FF0000', '#BC8F8F', '#4169E1', '#8B4513', '#FA8072', '#F4A460', '#2E8B57', '#FFF5EE', '#A0522D', '#C0C0C0', '#87CEEB', '#6A5ACD', '#708090', '#708090', '#FFFAFA', '#00FF7F', '#4682B4', '#D2B48C', '#008080', '#D8BFD8', '#FF6347', '#40E0D0', '#EE82EE', '#F5DEB3', '#FFFFFF', '#F5F5F5', '#FFFF00', '#9ACD32'] * 1000
    plt.figure(figsize = (14, 7))
    plt.title(name, fontsize = 20)
    aux = 0
    community_df = pd.DataFrame()
    print('show')
    for community in communities:
       nx.draw_networkx_nodes(G, pos, community, node_size = 20, node_color = colors[aux])
       aux = aux + 1

       for comm in community:
           community_df.loc[comm, 'community'] = aux
    
    community_df = community_df.reset_index().rename(columns = {'index': id_col})
    
    if path.exists(folder) == False:
        os.mkdir(folder)
    community_df.to_csv(folder + '/' + comm_file, sep = ';', index = False)
    
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.xlabel(text)
    plt.savefig(folder + '/' + image_file)


def find_save_communities(net, isolated = None, directed = False, id_col = 'Id', by_years = False, community_func = detect_communities_louvain,
                          largest_comp = False, core = False, folder = 'community', method_name = '', communities = None):
    """
    Find communities, show and save
    Args:
      net (DataFrame): network
      isolated (DataFrame): isolated vertices
      directed (bool): is graph directed
      id_col (str): Id column
      by_years (str): if True find communities by years 
      community_func (function): function to find communities
      largest_comp (bool): if True find communities in largest connected component
      core (int): if > 0 find comunities in core of graph
      folder (str): foler to save
      method_name (str): name of community detection method
      communities (list): list of comunities
    """
    net_no_periods = net.groupby([id_col + '1', id_col + '2']).sum()['weight'].reset_index()

    if path.exists(folder) == False:
        os.mkdir(folder)

    net_no_periods.to_csv(folder + '/' + 'net_comm.txt', sep = '\t', index = False, header = False)

    if directed: 
        G = nx.read_weighted_edgelist(folder + '/' + 'net_comm.txt', delimiter='\t', create_using=nx.DiGraph, nodetype = str)
    else:
        G = nx.read_weighted_edgelist(folder + '/' + 'net_comm.txt', delimiter='\t', nodetype = str)
    if isolated is not None:
        G.add_nodes_from(isolated[id_col])

    G.remove_edges_from(nx.selfloop_edges(G))    
    print('start')
    if largest_comp:
        if directed:
            components = nx.weakly_connected_components(G)
        else:
            components = nx.connected_components(G)
        comp_df = pd.DataFrame()
        i = 1
        for comp in components:
            for au in comp:
                comp_df.loc[au, 'component'] = i
            i += 1
                    
        comp_df = comp_df.reset_index().rename(columns = {'index': id_col})
        cnt_comp = comp_df.groupby('component').count().reset_index().rename(columns = {id_col:'count'})
        max_comp = cnt_comp[cnt_comp['count'] == np.max(cnt_comp['count'].values)]['component'].values[0]
        print('comp')
        S = G.subgraph(set(comp_df[comp_df['component'] == max_comp][id_col].values)).copy()
    else:
        S = G.copy()
    if core:
        S = nx.k_core(S, k = core)

    pos = nx.spring_layout(S)

    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', '#F0F8FF', '#FAEBD7', '#00FFFF', '#7FFFD4', '#F0FFFF', '#F5F5DC', '#FFE4C4', '#000000', '#FFEBCD', '#0000FF', '#8A2BE2', '#A52A2A', '#DEB887', '#5F9EA0', '#7FFF00', '#D2691E', '#FF7F50', '#6495ED', '#FFF8DC', '#DC143C', '#00FFFF', '#00008B', '#008B8B', '#B8860B', '#A9A9A9', '#006400', '#A9A9A9', '#BDB76B', '#8B008B', '#556B2F', '#FF8C00', '#9932CC', '#8B0000', '#E9967A', '#8FBC8F', '#483D8B', '#2F4F4F', '#2F4F4F', '#00CED1', '#9400D3', '#FF1493', '#00BFFF', '#696969', '#696969', '#1E90FF', '#B22222', '#FFFAF0', '#228B22', '#FF00FF', '#DCDCDC', '#F8F8FF', '#FFD700', '#DAA520', '#808080', '#008000', '#ADFF2F', '#808080', '#F0FFF0', '#FF69B4', '#CD5C5C', '#4B0082', '#FFFFF0', '#F0E68C', '#E6E6FA', '#FFF0F5', '#7CFC00', '#FFFACD', '#ADD8E6', '#F08080', '#E0FFFF', '#FAFAD2', '#D3D3D3', '#90EE90', '#D3D3D3', '#FFB6C1', '#FFA07A', '#20B2AA', '#87CEFA', '#778899', '#778899', '#B0C4DE', '#FFFFE0', '#00FF00', '#32CD32', '#FAF0E6', '#FF00FF', '#800000', '#66CDAA', '#0000CD', '#BA55D3', '#9370DB', '#3CB371', '#7B68EE', '#00FA9A', '#48D1CC', '#C71585', '#191970', '#F5FFFA', '#FFE4E1', '#FFE4B5', '#FFDEAD', '#000080', '#FDF5E6', '#808000', '#6B8E23', '#FFA500', '#FF4500', '#DA70D6', '#EEE8AA', '#98FB98', '#AFEEEE', '#DB7093', '#FFEFD5', '#FFDAB9', '#CD853F', '#FFC0CB', '#DDA0DD', '#B0E0E6', '#800080', '#663399', '#FF0000', '#BC8F8F', '#4169E1', '#8B4513', '#FA8072', '#F4A460', '#2E8B57', '#FFF5EE', '#A0522D', '#C0C0C0', '#87CEEB', '#6A5ACD', '#708090', '#708090', '#FFFAFA', '#00FF7F', '#4682B4', '#D2B48C', '#008080', '#D8BFD8', '#FF6347', '#40E0D0', '#EE82EE', '#F5DEB3', '#FFFFFF', '#F5F5F5', '#FFFF00', '#9ACD32'] * 1000
    
    if not by_years:
        if communities is None:
            communities = community_func(S)
        show_save_communities(S, communities, pos, name = 'Communities ' + method_name, id_col = id_col,
                          folder = folder, comm_file = 'communities.csv', image_file = 'image.png', text = f'{len(communities)} communities')
        return

    for year in range(2015, 2022):
        net_year = net[net['Y'] == year]
        net_year = net_year[(net_year[id_col + '1'].isin(list(map(int, S.nodes())))) | (net_year[id_col + '2'].isin(list(map(int, S.nodes()))))]
        
        net_year[[id_col + '1', id_col + '2', 'weight']].to_csv(folder + '/' + f'net_{year}.txt', sep = '\t', index = False, header = False)

        G_year = nx.read_weighted_edgelist(folder + '/' + f'net_{year}.txt', delimiter='\t', nodetype = str)
        if directed: 
            G_year = nx.read_weighted_edgelist(folder + '/' + f'net_{year}.txt', delimiter='\t', create_using=nx.DiGraph, nodetype = str)
        else:
            G_year = nx.read_weighted_edgelist(folder + '/' + f'net_{year}.txt', delimiter='\t', nodetype = str)

        if isolated is not None:
            isolated_year = isolated[isolated['Y'] == year]
            G_year.add_nodes_from(isolated_year[id_col]) 
        G_year.remove_edges_from(nx.selfloop_edges(G_year))   

        plt.figure(figsize = (15, 8))

        aux = 0
        communities = detect_communities_louvain(G_year)
        
        show_save_communities(G_year, communities, pos, name = 'Communities ' + method_name +  ' ' + str(year), id_col = id_col,
                          folder = folder, comm_file = f'communities_{year}.csv', image_file = f'image_{year}.png', text = f'{len(communities)} communities')
